## Prep

In [1]:
import numpy as np
import pandas as pd
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# ------- commutation matrix -------
def comm_mat(m, n):
    w = np.arange(m * n).reshape((m, n), order="F").T.ravel(order="F")
    return np.eye(m * n)[w, :]

# ------- matrix A(Sigma) -------
def create_A_Sigma(cov: np.ndarray, p: int) -> np.ndarray:
    identity = np.eye(p)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))

# ------- matrix A (intervention) -------
def create_A(A_Sigma: np.ndarray, mean: np.ndarray, p: int) -> np.ndarray:
    identity = mean[0] * np.eye(p)
    for i in range(1, p):
        identity = np.hstack((identity, mean[i] * np.eye(p)))
    return np.vstack((A_Sigma, identity))

num_exp = 10000 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix
b = 2
reg_param = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]

## 1 edge

<img src="Graphics/graph1.png" alt="Drawing" style="width: 100px;"/>

In [2]:
p = 2
seed = 0
index = 0

index_zero = np.array([[0, 1]])
support = np.array([0, 2, 3])
supp_compl = np.array([1])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_1edge = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_1edge.to_pickle("Experiments/DLL_DAG_1edge.pkl")

## 2 edges

### 3 nodes - Ex. 1

<img src="Graphics/graph2.png" alt="Drawing" style="width: 100px;"/>

In [3]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [1, 2], [2, 0]])
support = np.array([0, 3, 4, 7, 8])
supp_compl = np.array([1, 2, 5, 6])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_2edge_1 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_2edge_1.to_pickle("Experiments/DLL_DAG_2edge_1.pkl")

### 3 nodes - Ex. 2
<img src="Graphics/graph3.png" alt="Drawing" style="width: 100px;"/>

In [4]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [2, 0], [2, 1]])
support = np.array([0, 3, 4, 5, 8])
supp_compl = np.array([1, 2, 6, 7])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_2edge_2 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_2edge_2.to_pickle("Experiments/DLL_DAG_2edge_2.pkl")

### 3 nodes - Ex. 3
<img src="Graphics/graph4.png" alt="Drawing" style="width: 100px;"/>

In [5]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 0], [1, 2], [2, 0]])
support = np.array([0, 1, 4, 7, 8])
supp_compl = np.array([2, 3, 5, 6])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_2edge_3 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_2edge_3.to_pickle("Experiments/DLL_DAG_2edge_3.pkl")

## 3 edges

### 3 nodes - Ex. 1

<img src="Graphics/graph5.png" alt="Drawing" style="width: 100px;"/>

In [6]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [1, 2]])
support = np.array([0, 3, 4, 6, 7, 8])
supp_compl = np.array([1, 2, 5])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_1 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_1.to_pickle("Experiments/DLL_DAG_3edge_1.pkl")

### 4 nodes - Ex. 2

<img src="Graphics/graph6.png" alt="Drawing" style="width: 100px;"/>

In [7]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1]])
support = np.array([0, 4, 5, 9, 10, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11, 12, 13])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_2 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_2.to_pickle("Experiments/DLL_DAG_3edge_2.pkl")

### 4 nodes - Ex. 3
<img src="Graphics/graph7.png" alt="Drawing" style="width: 100px;"/>

In [8]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1]])
support = np.array([0, 1, 5, 9, 10, 14, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 8, 11, 12, 13])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_3 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_3.to_pickle("Experiments/DLL_DAG_3edge_3.pkl")

### 4 nodes - Ex. 4
<img src="Graphics/graph8.png" alt="Drawing" style="width: 100px;"/>

In [9]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 9, 10, 11, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11, 12, 13])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_4 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_4.to_pickle("Experiments/DLL_DAG_3edge_4.pkl")

### 4 nodes - Ex. 5
<img src="Graphics/graph9.png" alt="Drawing" style="width: 100px;"/>

In [10]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1]])
support = np.array([0, 1, 5, 9, 10, 14, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 8, 11, 12, 13])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_5 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_5.to_pickle("Experiments/DLL_DAG_3edge_5.pkl")

### 4 nodes - Ex. 6
<img src="Graphics/graph10.png" alt="Drawing" style="width: 100px;"/>

In [11]:
p = 4
seed = 0
index = 0

index_zero = np.array([[1, 0], [1, 2], [1, 3], [2, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 3, 5, 10, 15])
supp_compl = np.array([4, 6, 7, 8, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_6 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_6.to_pickle("Experiments/DLL_DAG_3edge_6.pkl")

### 4 nodes - Ex. 7
<img src="Graphics/graph11.png" alt="Drawing" style="width: 100px;"/>

In [12]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [1, 2], [1, 3], [2, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 2, 3, 4, 5, 10, 15])
supp_compl = np.array([1, 6, 7, 8, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_7 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_7.to_pickle("Experiments/DLL_DAG_3edge_7.pkl")

### 4 nodes - Ex. 8
<img src="Graphics/graph12.png" alt="Drawing" style="width: 100px;"/>

In [13]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 3, 4, 5, 8, 10, 15])
supp_compl = np.array([1, 2, 6, 7, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_8 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_8.to_pickle("Experiments/DLL_DAG_3edge_8.pkl")

### 4 nodes - Ex. 9
<img src="Graphics/graph13.png" alt="Drawing" style="width: 100px;"/>

In [14]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 8, 10, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 9, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_3edge_9 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_3edge_9.to_pickle("Experiments/DLL_DAG_3edge_9.pkl")

## 4 edges

### Ex. 1
<img src="Graphics/graph14.png" alt="Drawing" style="width: 100px;"/>

In [15]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 9, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11, 13])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_1 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_1.to_pickle("Experiments/DLL_DAG_4edge_1.pkl")

### Ex. 2
<img src="Graphics/graph15.png" alt="Drawing" style="width: 100px;"/>

In [16]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 0], [2, 1], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 6, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 7, 8, 9, 11, 13])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_2 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_2.to_pickle("Experiments/DLL_DAG_4edge_2.pkl")

### Ex. 3
<img src="Graphics/graph16.png" alt="Drawing" style="width: 100px;"/>

In [17]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 9, 10, 11, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_3 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_3.to_pickle("Experiments/DLL_DAG_4edge_3.pkl")

### Ex. 4
<img src="Graphics/graph17.png" alt="Drawing" style="width: 100px;"/>

In [18]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 8, 9, 10, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_4 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_4.to_pickle("Experiments/DLL_DAG_4edge_4.pkl")

### Ex. 5
<img src="Graphics/graph18.png" alt="Drawing" style="width: 100px;"/>

In [19]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 8, 9, 10, 12, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 9, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_5 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_5.to_pickle("Experiments/DLL_DAG_4edge_5.pkl")

### Ex. 6
<img src="Graphics/graph19.png" alt="Drawing" style="width: 100px;"/>

In [20]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 8, 10, 11, 12, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 9, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_6 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_6.to_pickle("Experiments/DLL_DAG_4edge_6.pkl")

### Ex. 7
<img src="Graphics/graph20.png" alt="Drawing" style="width: 100px;"/>

In [21]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 0], [1, 2], [1, 3], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 5, 8, 9, 10, 15])
supp_compl = np.array([2, 4, 6, 7, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_7 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_7.to_pickle("Experiments/DLL_DAG_4edge_7.pkl")

### Ex. 8
<img src="Graphics/graph21.png" alt="Drawing" style="width: 100px;"/>

In [22]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 5, 9, 10, 12, 15])
supp_compl = np.array([3, 4, 6, 7, 8, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_8 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_8.to_pickle("Experiments/DLL_DAG_4edge_8.pkl")

### Ex. 9
<img src="Graphics/graph22.png" alt="Drawing" style="width: 100px;"/>

In [23]:
p = 4
seed = 0
index = 0

index_zero = np.array([[1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 3, 5, 9, 10, 15])
supp_compl = np.array([4, 6, 7, 8, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_4edge_9 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_4edge_9.to_pickle("Experiments/DLL_DAG_4edge_9.pkl")

## 5 nodes

### Ex. 1
<img src="Graphics/graph23.png" alt="Drawing" style="width: 100px;"/>

In [24]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 6, 8, 10, 11, 12, 15])
supp_compl = np.array([1, 2, 3, 7, 9, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_5edge_1 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_5edge_1.to_pickle("Experiments/DLL_DAG_5edge_1.pkl")

### Ex. 2
<img src="Graphics/graph24.png" alt="Drawing" style="width: 100px;"/>

In [25]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 1], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 6, 8, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 7, 9, 11, 13])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_5edge_2 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_5edge_2.to_pickle("Experiments/DLL_DAG_5edge_2.pkl")

### Ex. 3
<img src="Graphics/graph25.png" alt="Drawing" style="width: 100px;"/>

In [26]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 8, 9, 10, 11, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_5edge_3 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_5edge_3.to_pickle("Experiments/DLL_DAG_5edge_3.pkl")

### Ex. 4
<img src="Graphics/graph26.png" alt="Drawing" style="width: 100px;"/>

In [27]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [2, 0], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 7, 9, 10, 11, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 8, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_5edge_4 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_5edge_4.to_pickle("Experiments/DLL_DAG_5edge_4.pkl")

### Ex. 5
<img src="Graphics/graph27.png" alt="Drawing" style="width: 100px;"/>

In [28]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 3]])
support = np.array([0, 4, 5, 9, 10, 12, 13, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_5edge_5 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_5edge_5.to_pickle("Experiments/DLL_DAG_5edge_5.pkl")

## 6 nodes
<img src="Graphics/graph28.png" alt="Drawing" style="width: 100px;"/>

In [29]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]])
support = np.array([0, 4, 5, 8, 9, 10, 12, 13, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 11])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_6edge = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_6edge.to_pickle("Experiments/DLL_DAG_6edge.pkl")